# Export metadata from OpenMetadata

In [1]:
from metadata.ingestion.ometa.ometa_api import OpenMetadata
from metadata.generated.schema.entity.services.connections.metadata.openMetadataConnection import (OpenMetadataConnection, AuthProvider)
from metadata.generated.schema.security.client.openMetadataJWTClientConfig import OpenMetadataJWTClientConfig

In [2]:
from creds import om_admin_token
server_config = OpenMetadataConnection(
    hostPort="http://datacatalog.casd.local/api",
    authProvider=AuthProvider.openmetadata,
    securityConfig=OpenMetadataJWTClientConfig(
        jwtToken=om_admin_token,
    ),
)
metadata = OpenMetadata(server_config)

In [3]:
# if it returns true, it means the connection is success 
metadata.health_check()

True

# Get entity by their fqn

We can get any entity from the openmetadata by using the fully qualified name.

In [18]:
from metadata.generated.schema.entity.data.table import Table

# the option up_depth and down_depth allows us to get the lineage information if it exists

def get_table_entity(om_connexion, table_fqn:str):
    return om_connexion.get_by_name(entity=Table, 
                                fqn=table_fqn)

table_entity = get_table_entity(metadata,"test-db-service.test-db.test-schema.test_user")

In [19]:
print(table_entity)

id=Uuid(__root__=UUID('d2739911-4301-41c9-9aed-9910d95c7abe')) name=EntityName(__root__='test_user') displayName=None fullyQualifiedName=FullyQualifiedEntityName(__root__='test-db-service.test-db.test-schema.test_user') description=None version=EntityVersion(__root__=0.1) updatedAt=Timestamp(__root__=1720081409210) updatedBy='ingestion-bot' href=Href(__root__=AnyUrl('http://datacatalog.casd.local/api/v1/tables/d2739911-4301-41c9-9aed-9910d95c7abe', scheme='http', host='datacatalog.casd.local', tld='local', host_type='domain', path='/api/v1/tables/d2739911-4301-41c9-9aed-9910d95c7abe')) tableType=None columns=[Column(name=ColumnName(__root__='id'), displayName=None, dataType=<DataType.BIGINT: 'BIGINT'>, arrayDataType=None, dataLength=None, precision=None, scale=None, dataTypeDisplay='bigint', description=Markdown(__root__='id of the user'), fullyQualifiedName=FullyQualifiedEntityName(__root__='test-db-service.test-db.test-schema.test_user.id'), tags=None, constraint=None, ordinalPositio

## get attributes of an entity

In [7]:
item=table_entity.id.__root__
print(f"content: {item}")
print(f"type: {type(item)}")

content: d2739911-4301-41c9-9aed-9910d95c7abe
content: <class 'uuid.UUID'>


## Get table lineage


In [21]:
def get_table_lineage(om_connexion, table_fqn:str, up_depth:int=1,down_depth:int=1):
    return om_connexion.get_lineage_by_name(
        entity=Table,
        fqn=table_fqn,
        # Tune this to control how far in the lineage graph to go
        up_depth=up_depth,
        down_depth=down_depth
    )
table_with_lineage= get_table_lineage(metadata,"test-db-service.test-db.test-schema.user_order_cube_detail")

In [25]:
print(table_with_lineage)

{'entity': {'id': 'fee8f96c-3e1e-4cbb-8992-dd69102bade4', 'type': 'table', 'name': 'user_order_cube_detail', 'fullyQualifiedName': 'test-db-service.test-db.test-schema.user_order_cube_detail', 'displayName': 'user_order_cube_detail', 'deleted': False, 'href': 'http://datacatalog.casd.local/api/v1/tables/fee8f96c-3e1e-4cbb-8992-dd69102bade4'}, 'nodes': [{'id': 'd2739911-4301-41c9-9aed-9910d95c7abe', 'type': 'table', 'name': 'test_user', 'fullyQualifiedName': 'test-db-service.test-db.test-schema.test_user', 'displayName': 'test_user', 'deleted': False, 'href': 'http://datacatalog.casd.local/api/v1/tables/d2739911-4301-41c9-9aed-9910d95c7abe'}, {'id': '6e53115b-0db2-4022-8138-463127ae57ce', 'type': 'table', 'name': 'test_order', 'fullyQualifiedName': 'test-db-service.test-db.test-schema.test_order', 'displayName': 'test_order', 'deleted': False, 'href': 'http://datacatalog.casd.local/api/v1/tables/6e53115b-0db2-4022-8138-463127ae57ce'}], 'upstreamEdges': [{'fromEntity': 'd2739911-4301-41c

In [ ]:
print(table_with_lineage['upstreamEdges'])